# Simulating IPFS and IPNS Systems

This notebook provides a Python-based simulation of IPFS (InterPlanetary File System) and IPNS (InterPlanetary Naming System) to test various linking strategies for storing and retrieving IPAROs.

The notebook uses three classes to simulate these systems:
- **IPARO**: Represents the storage object on IPFS.
- **IPNS**: Keeps track of the latest capture for different websites.
- **IPFS**: Simulates the hashing, storage, and retrieval of IPARO objects.

The goal of the simulation is to test various linking strategies.

In [1]:
# Importing the necessary libraries
from abc import ABC, abstractmethod
from dataclasses import dataclass
from datetime import datetime
from enum import Enum
from functools import cached_property
from math import floor
from typing import Optional
import hashlib
import random
import time

## IPARO Object

**Properties:**
- `URL`: The capture URL.
- `Timestamp`: The timestamp of the capture.
- `Content`: The contents of the IPARO.

**Functions:**
- `__str__`: Returns a string representation of the IPARO object.

In [2]:
@dataclass(frozen=True)
class IPARO:
    url: str
    content: bytes
    timestamp: datetime

    def __str__(self):
        """
        Returns a string representation of the IPARO object.

        Returns:
            str: A string containing the URL and content of the IPARO.
        """
        iparo = {
            "URL": self.url,
            "Content": self.content,
            "Timestamp": self.timestamp,
        }
        return str(iparo)


## The IPAROLink Object

**Description:**

- The IPAROLink class stores a link object, which will define links between two IPARO objects.
- The source CID is used to lookup the IPARO objects, so only the target CIDs are recorded in the IPAROLink object in the IPFS.

**Properties:**

- `seq_num`: The sequence number of the link.
- `timestamp`: The timestamp of the link.
- `cid`: The CID of the link.

In [3]:
@dataclass(frozen=True)
class IPAROLink:
    """
    Defines the IPARO Link class that has a sequence number, a datetime, and a CID.
    """
    seq_num: int
    timestamp: datetime
    cid: str


## The IPAROLinkCollection Object

**Description:**

- The IPAROLinkCollection class stores all the links from a source node.
- The source CID is used to lookup the IPARO objects, so only the target CIDs are recorded in the IPAROLink object.
- The previous link has the maximum sequence number of all the links.
    - This property must hold true since all strategies access the previous CID.
    - The current node and all nodes afterwards have not been hashed before the creation of the link collection so it isn't possible for any node to have a higher sequence number.

**Properties:**

- `links`: The list of all links coming from the source IPARO.
- `previous`: The link to the previous CID, which is the linked CID with the maximum sequence number.
- `first`: The link to the very first CID, which is the linked CID with sequence number 0.

**Methods:**
- `__str__`, `__repr__`: This should return the list of `IPAROLink` objects.
- `__len__`: This should return the total number of links in the collection.

In [4]:
class IPAROLinkCollection:
    """
    A helper class for linking IPAROs together. The two attributes are ``links`` (gets the links to other IPAROs),
    and ``iparo`` (the IPARO object to be linked).
    """

    def __init__(self, links: list[IPAROLink]):
        """
        Initializes the collection of links on the IPARO object.

        Params:
            links (list[IPAROLink]): The list of links.
        """
        self.links = links

        # The previous node is in the list of links (if there are any to begin with), and so would have the largest
        # sequence number.
        self.previous: IPAROLink | None = max(links, key=lambda link: link.seq_num) if len(links) > 0 else None

        first_links = [link for link in links if link.seq_num == 0]
        self.first: IPAROLink | None = first_links[0] if len(first_links) > 0 else None

    def __str__(self):
        return str(self.links)

    def __repr__(self):
        return str(self)
        
    def __len__(self):
        return len(self.links)

## IPNS Object

**Description:**
- The IPNS class stores and maps the latest CID of a website.
- Tracks the number of operations (get and update) performed.

**Functions:**
- `update`: Updates the latest CID of a website.
- `get_cid`: Retrieves the CID of the latest capture for a website.
- `get_counts`: Returns the number of operations performed.
- `reset_counts`: Resets the counters for operations.
- `reset`: Resets the data.
- `get_version_counts`: Get number of versions of a URL.

In [5]:
class IPNS:

    def __init__(self):
        """
        Initialize the IPNS object with an empty hashmap for storing CIDs
        and counters for tracking operations.
        """
        self.data: dict[str, str] = {}
        self.version_counts: dict[str, int] = {}
        self.update_count = 0
        self.get_count = 0

    def update(self, url: str, cid: str):
        """
        Updates the latest CID for a given URL.

        Args:
            url (str): The URL of the website.
            cid (str): The CID of the latest capture.
        """
        self.update_count += 1
        self.data[url] = cid
        self.version_counts[url] = self.version_counts.setdefault(url, 0) + 1

    def get_cid(self, url: str) -> Optional[str]:
        """
        Retrieves the latest CID for a given URL if it exists, else None.

        Args:
            url (str): The URL of the website.

        Returns:
            str: The CID of the latest capture for the given URL if it exists, else None.
        """
        self.get_count += 1
        return self.data.get(url)

    def get_number_of_nodes(self, url: str) -> int:
        """
        Retrieves the number of nodes in a given URL.

        Args:
            url (str): The URL of the website.

        Returns:
            int: The number of nodes for a given URL, or zero if the mapping is not present.
        """
        return self.version_counts.get(url, 0)

    def get_counts(self):
        """
        Returns the number of update and get operations performed.

        Returns:
            dict: Dictionary with the counts of update and get operations.
        """
        return {"get": self.get_count, "update": self.update_count}

    def reset_data(self):
        """
        Resets the data.
        """
        self.data: dict[str, str] = {}
        self.version_counts: dict[str, int] = {}

    def reset_counts(self):
        """
        Resets the operating counts. Used for the evaluation phase.
        """
        self.update_count = 0
        self.get_count = 0

## Link Strategy Parameters

**Description:** The LinkStrategyParams class is used to hold the parameters of a linking strategy that are usually needed for the LinkStrategy to properly work, without having to duplicate the code every time we create a linking strategy.

**Properties:**
- `url`: The URL, provided at the constructor level.
- `latest_cid`: The latest CID, which is equal to the CID of the URL (note that this is different from the newly created IPARO object), if it exists.
- `node_num`: The number of nodes created for the URL before the newly created IPARO object.
- `latest_node`: The node with the latest CID. Used for accessing the timestamp.
- `link`: The link to the latest node, if it exists.

In [6]:
class LinkStrategyParams:
    """
    This class is designed to add parameters from the IPNS and the IPFS that are determined at runtime.
    """

    def __init__(self, url: str):
        self.url = url
        self.node_num = ipns.get_number_of_nodes(url)

    @cached_property
    def latest_cid(self):
        return ipns.get_cid(self.url)

    @cached_property
    def latest_node(self):
        return ipfs.retrieve(self.latest_cid)

    @cached_property
    def latest_node_links(self):
        return ipfs.retrieve_links(self.latest_cid)

    @cached_property
    def link(self):
        return IPAROLink(seq_num=self.node_num - 1,
                         timestamp=self.latest_node.timestamp,
                         cid=self.latest_cid) if self.latest_node is not None else None


## Link Strategies
**Description:**
The LinkStrategy class encapsulates a linking strategy. This allows for more structure with regards to the approaches to evaluating them.

**Functions:**

- `get_linked_nodes`: Evaluate which IPARO nodes to link to, given the parameters for the linking strategy.

In [7]:
class LinkStrategy(ABC):

    @abstractmethod
    def get_linked_nodes(self, params: LinkStrategyParams) -> IPAROLinkCollection:
        pass

## Mode
**Description:**
The Mode class allows the user to specify the way that `retrieve_by_timestamp()` method yields a CID given a URL and a timestamp. Note that if there are no nodes (matching the relevant filters), the `retrieve_timestamp()` method will return `None`.

**Constants:**

- `LATEST_BEFORE`: Fetches the latest node that comes on or before the specifed timestamp and returns its CID (default).
- `EARLIEST_AFTER`: Fetches the latest node that comes on or before the specifed timestamp and returns its CID.
- `CLOSEST`: Fetches the latest node that comes closest to the specifed timestamp and returns its CID.

In [8]:
class Mode(Enum):
    LATEST_BEFORE = 0
    EARLIEST_AFTER = 1
    CLOSEST = 2

## IPFS Object

**Description:**
- The IPFS class stores the nodes and simulates the hashing, storage, and retrieval operations.
- Tracks the number of operations (hash, store, retrieve).

**Functions:**
- `hash`: Hashes the content of a node to generate its CID.
- `link`: Links a node with other nodes using a linking strategy.
- `store`: Stores a node with its CID.
- `retrieve`: Retrieves a node using its CID.
- `retrieve_links`: Retrieves the list of all links using the node's CID.
- `retrieve_by_number`: Retrieves a node with a URL and its sequence number.
- `retrieve_by_date`: Retrieves a node with a URL and its date, according to a given mode.
- `get_counts`: Returns the number of operations performed.
- `reset_counts`: Resets the counters for operations.

In [9]:
class IPFS:
    """
    The InterPlanetary File System is responsible for hashing, storing,
    retrieving, and linking IPARO objects.
    """

    def __init__(self):
        self.data: dict[str, IPARO] = {}
        self.links: dict[str, IPAROLinkCollection] = {}
        self.hash_count = 0
        self.retrieve_count = 0
        self.store_count = 0

    def hash(self, iparo: IPARO) -> str:
        """
        Hashes the IPARO to generate a CID.

        Args:
            iparo: IPARO: The IPARO object.

        Returns:
            str: The generated CID.
        """
        self.hash_count += 1
        iparo_string = str(iparo)
        sha256_hash = hashlib.sha256(iparo_string.encode()).hexdigest()
        return 'Qm' + sha256_hash[:34]

    def store(self, iparo: IPARO) -> str:
        """
        Stores a node with its CID.

        Args:
            iparo (IPARO): The IPARO object to store.

        Returns:
            The CID of the newly stored IPARO.
        """
        cid = self.hash(iparo)
        self.store_count += 1
        self.data[cid] = iparo
        return cid

    def link(self, cid: str, strategy: LinkStrategy, params: LinkStrategyParams) -> IPAROLinkCollection:
        """
        Links an IPARO with the specified CID to other IPAROs, according to the provided linking strategy.

        Args:
            cid (str): The CID of the IPARO object.
            strategy (LinkStrategy): The link strategy to use when linking to other IPAROs.
            params (LinkStrategyParams): The parameters for the linking strategy.

        Returns:
            IPAROLinkCollection: The links added for the newly created node.
        """
        links = strategy.get_linked_nodes(params)
        self.links[cid] = links
        return links

    def reset_data(self):
        self.data: dict[str, IPARO] = {}
        self.links: dict[str, IPAROLinkCollection] = {}

    def retrieve(self, cid) -> Optional[IPARO]:
        """
        Retrieves the IPARO object corresponding to a given CID, if it exists, otherwise, ``None``.
        """
        self.retrieve_count += 1
        return self.data.get(cid)

    def retrieve_links(self, cid: str) -> IPAROLinkCollection:
        """
        Retrieves the IPARO links corresponding to a given CID.
        """
        self.retrieve_count += 1
        return self.links.get(cid, IPAROLinkCollection([]))

    def retrieve_by_timestamp(self, url: str, target_timestamp: datetime, mode: Mode = Mode.LATEST_BEFORE) -> \
            Optional[str]:
        """
        Retrieves the IPARO versions of a given URL closest to a given datetime if at least one
        IPARO version is stored in the IPFS, otherwise ``None``. Default is the latest timestamp
        that occurs before the target timestamp, but ``Mode.EARLIEST_AFTER`` gives the node with
        the earliest time after a given timestamp, and ``Mode.CLOSEST`` gives the node with the
        closest timestamp to a given timestamp. If the distance from the two closest times to
        the target timetamp are equal, the CID with the earlier timestamp will be chosen.
        """
        self.retrieve_count += 1

        timestamps = sorted((iparo.timestamp, cid) for cid, iparo in self.data.items() if iparo.url == url)

        # Early exit
        if len(timestamps) == 0:
            return None

        timestamp: Optional[datetime]
        if mode == Mode.LATEST_BEFORE:
            timestamps = [t for t in timestamps if t[0] <= target_timestamp]
            _, cid = min(timestamps, default=None, key=lambda ts: target_timestamp - ts[0])
        elif mode == Mode.CLOSEST:
            # Find the key with the minimum difference to the target timestamps
            _, cid = min(timestamps, default=None, key=lambda ts: abs(ts[0] - target_timestamp))
        else:
            timestamps = [t for t in timestamps if t[0] >= target_timestamp]
            _, cid = min(timestamps, default=None, key=lambda ts: ts[0] - target_timestamp)

        return cid

    def retrieve_by_number(self, url: str, number: int) -> str:
        """
        Retrieves the IPARO CID corresponding to a given sequence number and a URL.
        """
        retrieval_strategy = NumberRetrievalStrategy(url, number)
        cid = retrieval_strategy.retrieve()
        return cid

    def get_counts(self) -> dict:
        """
        Returns the number of hash, store, and retrieve operations performed.

        Returns:
            dict: Dictionary with counts of hash, store, and retrieve operations.
        """
        counts = {"hash": self.hash_count, "store": self.store_count,
                  "retrieve": self.retrieve_count}
        return counts

    def reset_counts(self):
        """
        Resets the operation counters.
        """
        self.hash_count = 0
        self.store_count = 0
        self.retrieve_count = 0

    def get_all_cids(self, url: str) -> list[str]:
        """
        Retrieves the list of all CIDs and IPAROs in the IPFS, corresponding to the given URL.
        The nodes are sorted from latest to earliest.
        """
        cids = []
        cid = ipns.get_cid(url)
        while True:
            if cid is not None:
                cids.append(cid)
            links = self.retrieve_links(cid)
            if links is None or links.previous is None:
                break
            cid = links.previous.cid

        return cids

## IPAROFactory

**Description**: The purpose of the IPAROFactory class is to create IPARO nodes.

**Methods:**
- `create_node`: A method that takes in two arguments and creates an IPARO object out of the URL and the content.

In [10]:
class IPAROFactory:
    @classmethod
    def create_node(cls, url: str, content: bytes) -> IPARO:
        """
        Creates an IPARO object.

        Args:
            url (str): The URL of the IPARO object.
            content (bytes): The contents of the IPARO object.
        """
        timestamp = datetime.now()
        iparo = IPARO(url=url, content=content, timestamp=timestamp)
        return iparo


## RetrievalStrategy (WIP)

**Description**: The purpose of the RetrievalStrategy class is to find the optimal links to retrieve in order to find the correct node.

**Methods**:
- `retrieve`: Given the parameters for the RetrievalStrategy class, retrieves the CID that is the "least distant" by a given metric, or `None` if there is no such link.

In [11]:
class RetrievalStrategy(ABC):

    @abstractmethod
    def retrieve(self) -> Optional[IPAROLink]:
        pass


class NumberRetrievalStrategy(RetrievalStrategy):

    def __init__(self, url: str, target_num: int):
        self.target_num = target_num
        self.url = url

    def retrieve(self) -> Optional[str]:
        cid = ipns.get_cid(self.url)
        if self.target_num == ipns.get_number_of_nodes(self.url) - 1:
            return cid
        while True:
            relevant_links = [link for link in ipfs.retrieve_links(cid).links if link.seq_num >= self.target_num]
            link = min(relevant_links, default=None, key=lambda link: link.seq_num)
            # Works because if link is None, the code will return None, but it will return the
            # target node if found.
            if link is None:
                return None
            elif link.seq_num == self.target_num:
                return link.cid

            cid = link.cid

## Initialization and Operation Tracking

Here, we initialize the IPFS and IPNS objects and define a helper function `get_op_counts()` to display the number of operations performed. Additionally, the `initialize()` operation sets up those objects and the `automate_node_creation()` function takes in a URL, a number of nodes, and a `LinkStrategy` and returns a list of IPARO objects to debug.


In [12]:
def initialize():
    '''
    Initializes the simulated IPFS and IPNS.

    (This function should be in a separate file, but for the sake of simplicity, it is included here.)
    '''
    global ipfs, ipns, iparo_factory
    ipfs = IPFS()
    ipns = IPNS()
    iparo_factory = IPAROFactory()

# # Initializing the simulated IPFS and IPNS
# ipfs = IPFS()
# ipns = IPNS()

def get_op_counts():
    '''
    Displays the number of operations performed by IPNS and IPFS.
    '''
    print("Number of operations IPNS performed:")
    print(ipns.get_counts())
    print("Number of operations IPFS performed:")
    print(ipfs.get_counts())

def setup():
    '''
    Sets up the testing environment for the different linking strategies.
    '''
    ipns.reset_data()
    ipns.reset_counts()
    ipfs.reset_data()
    ipfs.reset_counts()


def test_storage_strategy(strategy: LinkStrategy, url: str, node_num: int):
    setup()
    # Automate the creation of additional nodes
    for i in range(node_num):
        content = f"Node {i}"
        iparo = IPAROFactory.create_node(url, content)
        cid = ipfs.store(iparo)
        params = LinkStrategyParams(url)
        link_col = ipfs.link(cid, strategy, params)
        ipns.update(url, cid)
        time.sleep(0.01)
    get_op_counts()  # Output operation counts

def test_retrieval_strategy(url: str, node_num: int):
    # Reset the operation counts
    ipns.reset_counts()
    ipfs.reset_counts()
    
    # Pick a random node to search for
    node_num = random.randint(0, node_num - 1)
    print(f"Looking for node with sequence number: {node_num}")
    
    # Traverse back through the linked nodes to find the target
    retrieval_strategy = NumberRetrievalStrategy(url, node_num)
    cid = retrieval_strategy.retrieve()
    # Output the found node
    if cid is not None:
        node = ipfs.retrieve(cid)
        print(f"Node found with contents: {node.content}")
    else:
        print("Can't find node")
    
    get_op_counts()
    
initialize()

## Testing Different Linking Strategies

### 1. Linking to Only the Previous Node

In this test, each node will link only to the previous node in the chain. This strategy will be used to simulate a simple sequential storage system.


In [13]:
class SingleStrategy(LinkStrategy):

    def get_linked_nodes(self, params: LinkStrategyParams) -> IPAROLinkCollection:
        links = [params.link] if params.latest_node is not None else []
        return IPAROLinkCollection(links)

#### Storing Nodes

In [14]:
URL = "https://www.example.com/"
NODE_NUM = 100
# The beautiful thing about this method is it automatically does the setup and cleanup for you!
test_storage_strategy(SingleStrategy(), URL, NODE_NUM)

Number of operations IPNS performed:
{'get': 100, 'update': 100}
Number of operations IPFS performed:
{'hash': 100, 'store': 100, 'retrieve': 100}


By adding a `LinkStrategy` class and a method to store all the nodes, we managed to cut down the number of lines of code significantly. How awesome is that!

#### Retrieving Nodes

The following section tests retrieval of nodes by simulating a random node search.


In [15]:
test_retrieval_strategy(URL, NODE_NUM) # Wow! Only one line of code is necessary to test retrieval by sequence number!

Looking for node with sequence number: 31
Node found with contents: Node 31
Number of operations IPNS performed:
{'get': 1, 'update': 0}
Number of operations IPFS performed:
{'hash': 0, 'store': 0, 'retrieve': 69}


As you can see, the number of retrieve operations (denoted by $R$), on average, is $E[R]=O(n)$ for $n$ equal to the number of nodes. You can check this by noting that there is a $1/n$ chance of picking any number from nodes 0 to $n-1$, meaning we need from 1 to $n$ retrievals. Therefore, if we sum up the first $n$ numbers, we get $n(n+1)/2$, which means the average number of retrievals is $(n+1)/2$. The worst case happens when we try to get node 0 (the very first node), in which case the number of retrievals required is $n$. However, the number of links required for this setup is only equal to $n-1$. If we cannot find the link, all the nodes must be traversed.

### 2. Linking to all previous nodes

In this test, each node will link to all the previous nodes in the chain.

In [16]:
class ComprehensiveStrategy(LinkStrategy):

    def get_linked_nodes(self, params: LinkStrategyParams) -> IPAROLinkCollection:
        if params.node_num == 0:
            return IPAROLinkCollection([])
        cid = params.latest_cid
        links = [params.link]
        while True:
            previous_links = ipfs.retrieve_links(cid)
            previous_link = previous_links.previous
            if previous_link is None:
                break
            links.append(previous_link)
            cid = previous_link.cid
        return IPAROLinkCollection(links)

#### Storing Nodes

From here, there are 2 ways of creating a new node for this linking strategy, since this is a simulation, no data corruption can happen but that might not be true in practice. When retrieving the latest node which should contain the CIDs of all the previous node, two scenarios can happen:
1. The data is intact and the CIDs in the list is "correct" (which we really can't know for sure) and we can just add it to the new node we're creating
2. The data is corrupt and one or more of the CIDs is wrong or unfinished, in which case we have to recheck every CID to rebuild a new list of linked CIDs (not to mention fixing all the corrupted nodes)

So, for the purpose of this simulation, we will perform a check for every CID to simulate the worst case scenario every time

In [17]:
# Testing parameters
NODE_NUM = 100
URL = "example.com"
test_storage_strategy(ComprehensiveStrategy(), URL, NODE_NUM)

Number of operations IPNS performed:
{'get': 99, 'update': 100}
Number of operations IPFS performed:
{'hash': 100, 'store': 100, 'retrieve': 5049}


In this worst case scenario, where we have to retrieve and verify every CIDs in the linked CIDs of an IPARO, the retrieve count goes to more than 5000 (if we're storing 100 nodes). We count link retrievals as one retrieval each, and we need to all the CIDs, which is why the cost is that high.\
Of course this trade off makes it really easy to navigate to all the nodes just from the latest nodes, as you can see from the next section.

#### Retrieving nodes

The following section tests retrieval of nodes by simulating a random node search.

In [18]:
test_retrieval_strategy(URL, NODE_NUM)

Looking for node with sequence number: 81
Node found with contents: Node 81
Number of operations IPNS performed:
{'get': 1, 'update': 0}
Number of operations IPFS performed:
{'hash': 0, 'store': 0, 'retrieve': 2}


The number of IPFS retrievals drops down to 1. This is because the links connect to all the nodes. Of course, this has a tradeoff of being not only hard to build (costs $n(n-1)/2$ IPFS retrievals to set up) but also space-inefficient (needs $n(n-1)/2$ links).

### 3. Linking to previous and first node

In this test, each node will link to the previous node and the first node in the chain.

In [19]:
class KPreviousStrategy(LinkStrategy):

    def __init__(self, k: int):
        self.k = k

    def get_linked_nodes(self, params: LinkStrategyParams) -> IPAROLinkCollection:
        first_link: Optional[IPAROLink] = params.latest_node_links.first

        if params.latest_cid is None:
            # Case 1: There are no nodes, so no links.
            return IPAROLinkCollection([])
        elif first_link is None:
            # Case 2: There is one node, but first_link has not been updated yet. This means the latest CID is the
            # first ID
            return IPAROLinkCollection([IPAROLink(seq_num=0,
                                                  timestamp=params.latest_node.timestamp, cid=params.latest_cid)])
        cid = params.latest_cid
        links = [first_link, params.link]

        i = 1
        while i < self.k:
            previous = ipfs.retrieve_links(cid).previous
            if previous.cid == first_link.cid:
                break
            links.append(previous)
            cid = previous.cid
            i += 1
        return IPAROLinkCollection(links)

class PreviousStrategy(KPreviousStrategy):

    def __init__(self):
        super().__init__(1)

#### Storing Nodes

In [20]:
# The beautiful thing about this method is it automatically does the setup and cleanup for you!
NODE_NUM = 100
URL = "example.com"

test_storage_strategy(PreviousStrategy(), URL, NODE_NUM)

Number of operations IPNS performed:
{'get': 100, 'update': 100}
Number of operations IPFS performed:
{'hash': 100, 'store': 100, 'retrieve': 199}


#### Retrieving Nodes

In [21]:
test_retrieval_strategy(URL, NODE_NUM)

Looking for node with sequence number: 76
Node found with contents: Node 76
Number of operations IPNS performed:
{'get': 1, 'update': 0}
Number of operations IPFS performed:
{'hash': 0, 'store': 0, 'retrieve': 24}


Linking to previous and first node does not help much with the number of retrievals. The worst case scenario, if the node is guaranteed to be in the IPFS, happens precisely when node 1 is the node to search, which means it takes $n-1$ retrievals. Assuming a uniform distribution, the average number of retrievals $E[R]$ is cut down by $1-1/n$ because there are $n-1$ fewer operations to search for node 0 (i.e. from $n$ to 1, but the other $n-1$ nodes take the same number of operations to find.

### 4. Linking to K-previous and first node

In this test, each node will link to K previous node and the first node in the chain.

#### Storing Nodes

In [22]:
# Testing parameters
NODE_NUM = 100
URL = "example.com"
K = 5

test_storage_strategy(KPreviousStrategy(K), URL, NODE_NUM)

Number of operations IPNS performed:
{'get': 100, 'update': 100}
Number of operations IPFS performed:
{'hash': 100, 'store': 100, 'retrieve': 585}


#### Retrieving Nodes

In [23]:
test_retrieval_strategy(URL, NODE_NUM)

Looking for node with sequence number: 51
Node found with contents: Node 51
Number of operations IPNS performed:
{'get': 1, 'update': 0}
Number of operations IPFS performed:
{'hash': 0, 'store': 0, 'retrieve': 11}


In the worst case, we would expect to take $n/k+O(1)$ retrieval operations, which happens if the node to find has sequence number between 1 and $n-k\lfloor n/k\rfloor$. The expected number of retrieval operations is not much better asymptotically: it takes (on average) $n/(2k)+O(1)$ operations, assuming a uniform distribution.

### 5. Linking to K-random and first node

In this test, each node will link to a random K previous node and the first node in the chain. We could use a heuristic based on the closest target to the target node (given its sequence number).

In [24]:
class KRandomStrategy(LinkStrategy):

    def __init__(self, k_min: int, k_max: int):
        self.k_min = k_min
        self.k_max = k_max

    def get_linked_nodes(self, params: LinkStrategyParams) -> IPAROLinkCollection:
        k = random.randint(self.k_min, self.k_max)
        # Check if the number of linked CIDs is greater than K and add K-1 random linked CIDs
        latest_node_links = params.latest_node_links.links
        links = [latest_node_links[0]] if len(latest_node_links) > 0 else []
        if len(latest_node_links) > k:
            links.extend(random.sample(latest_node_links[1:], k - 1))
        # If the number of linked CIDs is less than K add all the linked CIDs
        else:
            links.extend(latest_node_links[1:])

        if params.link is not None:
            links.append(params.link)

        return IPAROLinkCollection(links)

#### Storing Nodes

In [25]:
import random

# Testing parameters
NODE_NUM = 100
URL = "example.com"
Kmin = 5
Kmax = 10

test_storage_strategy(KRandomStrategy(Kmin, Kmax), URL, NODE_NUM)

Number of operations IPNS performed:
{'get': 100, 'update': 100}
Number of operations IPFS performed:
{'hash': 100, 'store': 100, 'retrieve': 200}


#### Retrieving nodes

In [26]:
test_retrieval_strategy(URL, NODE_NUM)

Looking for node with sequence number: 94
Node found with contents: Node 94
Number of operations IPNS performed:
{'get': 1, 'update': 0}
Number of operations IPFS performed:
{'hash': 0, 'store': 0, 'retrieve': 3}


The theoretical worst-case scenario for K-random would be if all the random links (for nodes indexed $k+1$ to $n$) point to the first $k$ nodes and the node to find has index $k+1$. Then, it would take $n-(k+1)$ hops to find it. But on average, we can try forming a recurrence relation. Let $p$ be the probability that there is at least one link with sequence number between $m$ and $n-2$. If we assume that $d$ is the difference between the two indices $n$ and $m$, where $m$ is the target node index and $n$ is the current node index, then the probability a uniformly distributed random node has sequence number between $m$ and $n-2$ is $\frac{n-m-2}{n}=1-\frac{m+2}{n}$.

For the average case scenario, with the assumption that the distribution is uniform, we can perhaps try modeling the expected amount of time. There are $k$ randomly distributed nodes, so the probability that there are *no* links with sequence number between $m$ and $n-2$ is $1-p=\left(1-\frac{n-m+2}{n}\right)^k=\left(1-\frac{d+2}{n}\right)^k$. Taking the complement yields $p=1-\left(1-\frac{m+2}{n}\right)^k.$ This means the expectation of the distance traveled $D$ is 
$$E[D]=\left(1-\frac{d+2}{n}\right)^k+\left[1-\left(1-\frac{d+2}{n}\right)^k\right](d/2).$$
This means when $d+2=cn$ for some $0<c<1$ (equivalently, $d=O(n)$), then we have $E[D]=(1-(1-c)^k)(d/2)+(1-c)^k=O(n)$, which means on average, $T(d)=1+T((1-(1-c)^k/2)d)$. In other words, at the beginning, when the distance is large, it seems like the average-case requires logarithmic time. But then at $d=O(\sqrt{n})$, $E[D]$ now becomes $(1-1/\sqrt{n})^k+[1-(1-1/\sqrt{n})^k]\approx (1-1/\sqrt{n})^k+(1-(1-k/\sqrt{n}+O(1/n)))\sqrt{n}/2$, which is equal to
$(1-1/\sqrt{n})^k+(k/\sqrt{n}+O(1/n))\sqrt{n}/2=(1-1/\sqrt{n})^k+k/2=O(1)$ as $n\to\infty$ since $k=O(1)$.

That means when $d=O(\sqrt{n})$, the recurrence relation of the expected amount of time is linear: $T(d)=1+T(d-O(1))$. The math can be hard to understand, but the implication is that the average amount of time to access a random node with K-random nodes is given by $O(\sqrt{n})+O(\log{n})=O(\sqrt{n})$ since $\sqrt{n}$ dominates $\log{n}$.

### 6. Linking to Sequential Exponential (Base K)

In [27]:
class SequentialExponentialStrategy(LinkStrategy):

    def __init__(self, k: float):
        self.k = k

    def get_linked_nodes(self, params: LinkStrategyParams) -> IPAROLinkCollection:

        if params.node_num < 3:
            return ComprehensiveStrategy().get_linked_nodes(params)
        # Required: node_num >= 2
        indices: set[int] = {0, params.node_num - 1}
        index = 1.0
        while index < params.node_num:
            indices.add(params.node_num - floor(index))
            index *= self.k

        links = []

        for i in indices:
            cid = ipfs.retrieve_by_number(params.url, i)
            node = ipfs.retrieve(cid)
            link = IPAROLink(seq_num=i, timestamp=node.timestamp, cid=cid)
            links.append(link)

        return IPAROLinkCollection(links)

#### Storing nodes

In [28]:
# Test Parameters
K = 2
URL = "example.com"
NODE_NUM = 50

test_storage_strategy(SequentialExponentialStrategy(K), URL, NODE_NUM)

Number of operations IPNS performed:
{'get': 279, 'update': 50}
Number of operations IPFS performed:
{'hash': 50, 'store': 50, 'retrieve': 806}


As you can see, the number of retrievals required for the IPFS is less than that of the comprehensive strategy, but greater than that of the single and K-previous strategies. In fact, the number of links $L$ is
$$L\approx \sum_{i=1}^n{O(\log_k{(i)})}=O(n\log{n})$$
We could use a heuristic based on the closest target, which for two indices $m$ and $n$ means only $O(\log{(\max{(m, n)})})$ hops are necessary in the worst case scenario.

In [29]:
test_retrieval_strategy(URL, NODE_NUM)

Looking for node with sequence number: 49
Node found with contents: Node 49
Number of operations IPNS performed:
{'get': 1, 'update': 0}
Number of operations IPFS performed:
{'hash': 0, 'store': 0, 'retrieve': 1}


### 7. Linking to sequentially uniform N-prior

In [33]:
class SequentialUniformNPriorStrategy(LinkStrategy):

    def __init__(self, n: int):
        self.n = n

    def get_linked_nodes(self, params: LinkStrategyParams) -> IPAROLinkCollection:
        # Special case: No nodes in collection
        if params.node_num == 0:
            return IPAROLinkCollection([])
        indices: set[int] = {(params.node_num - 1) * i // self.n for i in range(self.n + 1)}

        links = []

        for i in indices:
            cid = ipfs.retrieve_by_number(params.url, i)
            node = ipfs.retrieve(cid)
            link = IPAROLink(seq_num=i, timestamp=node.timestamp, cid=cid)
            links.append(link)

        return IPAROLinkCollection(links)

#### Storing Nodes

In [34]:
# Test Parameters
num_links = 10
URL = "example.com"
NODE_NUM = 50

test_storage_strategy(SequentialUniformNPriorStrategy(num_links), URL, NODE_NUM)

Number of operations IPNS performed:
{'get': 484, 'update': 50}
Number of operations IPFS performed:
{'hash': 50, 'store': 50, 'retrieve': 1224}


#### Retrieving Nodes

In [152]:
test_retrieval_strategy(URL, NODE_NUM)

Looking for node with sequence number: 41
Node found with contents: Node 41
Number of operations IPNS performed:
{'get': 1, 'update': 0}
Number of operations IPFS performed:
{'hash': 0, 'store': 0, 'retrieve': 4}


The number of retrievals required for sequential uniform (assuming $k$ prior nodes are selected out of $n$ total nodes) is at most $n/k+O(1)$ where $k$ is the number of prior nodes, and this can happen when we want to find node with sequence number one greater than the sequence number of the nearest link.

### Other strategies to be tested